# Notebook Title

## Setup Python and R environment
you can ignore this section

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [5]:
%%R

# Import data with R
df <- read_csv('311_processed.csv', show_col_types = FALSE)
df %>% head(4)

# A tibble: 4 × 52
  `Unique Key` `Created Date`         `Closed Date`         Agency `Agency Name`
         <dbl> <chr>                  <chr>                 <chr>  <chr>        
1     59892199 12/31/2023 11:38:00 PM 01/05/2024 07:45:00 … DEP    Department o…
2     59889491 12/31/2023 11:17:00 PM 01/05/2024 07:30:00 … DEP    Department o…
3     59893098 12/31/2023 11:08:00 PM 01/05/2024 07:32:00 … DEP    Department o…
4     59886784 12/31/2023 10:53:00 PM 01/03/2024 09:24:00 … DEP    Department o…
# ℹ 47 more variables: `Complaint Type` <chr>, Descriptor <chr>,
#   `Location Type` <lgl>, `Incident Zip` <dbl>, `Incident Address` <chr>,
#   `Street Name` <chr>, `Cross Street 1` <chr>, `Cross Street 2` <chr>,
#   `Intersection Street 1` <lgl>, `Intersection Street 2` <lgl>,
#   `Address Type` <chr>, City <chr>, Landmark <lgl>, `Facility Type` <lgl>,
#   Status <chr>, `Due Date` <lgl>, `Resolution Description` <chr>,
#   `Resolution Action Updated Date` <chr>, `Community Board` <chr>, …


In addition: Warning message:
One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat) 


## 👉 Grab Census Data

1. loading the Census API key

In [6]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


False

In [8]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [9]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 46 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06011PR_001 Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 2 B06011PR_002 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 3 B06011PR_003 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 4 B06011PR_004 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 5 B06011PR_005 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 6 B06011_001   Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 7 B06011_002   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 8 B06011_003   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 9 B06011_004   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
10 B06011_005   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
11 B07011

In [10]:
%%R 
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
nyc_census_data <- get_acs(geography = "tract", 
                      state='NY',
                      county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                      variables = c(
                        population="B01003_001",
                        med_inc="B19013_001"
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=T)

nyc_census_data

  |======================================================================| 100%
Simple feature collection with 4654 features and 5 fields (with 2 geometries empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -74.25609 ymin: 40.4961 xmax: -73.70036 ymax: 40.91771
Geodetic CRS:  NAD83
First 10 features:
         GEOID                                       NAME   variable estimate
1  36081014700  Census Tract 147, Queens County, New York population     2863
2  36081014700  Census Tract 147, Queens County, New York    med_inc    71815
3  36047058400   Census Tract 584, Kings County, New York population     3655
4  36047058400   Census Tract 584, Kings County, New York    med_inc    67315
5  36061006900 Census Tract 69, New York County, New York population     2450
6  36061006900 Census Tract 69, New York County, New York    med_inc   237500
7  36047073000   Census Tract 730, Kings County, New York population     2676
8  36047073000   Census Tract 730, Kings County, N

Getting data from the 2017-2021 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'
In addition: Warning message:
• You have not set a Census API key. Users without a key are limited to 500
queries per day and may experience performance limitations.
ℹ For best results, get a Census API key at
http://api.census.gov/data/key_signup.html and then supply the key to the
`census_api_key()` function to use it throughout your tidycensus session.
This warning is displayed once per session. 


## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [12]:
%%R

# Merge the 311 data with the census data
df <- df %>% 
    left_join(nyc_census_data, by = c("TRACT" = "GEOID")) %>% 
    select(-geometry) # remove the geometry column

df %>% head(4)

# A tibble: 4 × 56
  `Unique Key` `Created Date`         `Closed Date`         Agency `Agency Name`
         <dbl> <chr>                  <chr>                 <chr>  <chr>        
1     59892199 12/31/2023 11:38:00 PM 01/05/2024 07:45:00 … DEP    Department o…
2     59889491 12/31/2023 11:17:00 PM 01/05/2024 07:30:00 … DEP    Department o…
3     59893098 12/31/2023 11:08:00 PM 01/05/2024 07:32:00 … DEP    Department o…
4     59886784 12/31/2023 10:53:00 PM 01/03/2024 09:24:00 … DEP    Department o…
# ℹ 51 more variables: `Complaint Type` <chr>, Descriptor <chr>,
#   `Location Type` <lgl>, `Incident Zip` <dbl>, `Incident Address` <chr>,
#   `Street Name` <chr>, `Cross Street 1` <chr>, `Cross Street 2` <chr>,
#   `Intersection Street 1` <lgl>, `Intersection Street 2` <lgl>,
#   `Address Type` <chr>, City <chr>, Landmark <lgl>, `Facility Type` <lgl>,
#   Status <chr>, `Due Date` <lgl>, `Resolution Description` <chr>,
#   `Resolution Action Updated Date` <chr>, `Community Board` <chr>, …


In [13]:
%%R

#save df to csv

write_csv(df, '311_census.csv')